In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Clustering
using Distances
using Iterators

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [2]:
db = DB("data/corpora/wordnet/")

WordNet.DB

In [3]:
lemma = db['n', "state"]
s1,s2,s3,s4 = synsets(db, lemma)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


8-element Array{WordNet.Synset,1}:
 (n) province, state (the territory occupied by one of the constituent administrative districts of a nation; "his state is in the deep south")                                                                                                                                                                            
 (n) state (the way something is with respect to its main attributes; "the current state of knowledge"; "his state of health"; "in a weak financial state")                                                                                                                                                               
 (n) state (the group of people comprising the government of a sovereign state; "the state has lowered its income tax")                                                                                                                                                                                                   
 (n) res publica, la

In [4]:
s1.

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


LoadError: LoadError: syntax: incomplete: premature end of input
while loading In[4], in expression starting on line 1

In [5]:
ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in _collect(::Array{SubString{String},1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:258
 in #error_content#12(::Symbol, ::String, ::Function, ::LoadError, ::Array{Ptr{Void},1}) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:75
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:238
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##24#30)() at ./task.jl:309
while loading /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl, in expression starting on line 31
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in collect_to!(::Array{String,1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Int64, ::Int64) at ./array.jl:277
 in _collect(:

In [6]:
function weighted_average(logprobs, embeddings)
    function normal_probs(logprobs::Vector)
        ret = copy(logprobs)
        max_lp = maximum(logprobs)
        ret.-=max_lp #Bring closer to zero
        map!(exp,ret)
        denom = sum(ret)
        ret./=denom
    end
    
    
    ret = zeros(size(embeddings[1]))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight*embedding
    end
    ret
end

function all_logprobs_of_context(embed, context, middle_word)
    [Query.logprob_of_context(embed, context,wv; skip_oov=true, normalise_over_length=true)
        for wv in embed.embedding[middle_word]]
end

function synthesize_embedding(ee,context, middle_word)
    logprobs = all_logprobs_of_context(ee, context,middle_word)
    weighted_average(logprobs, ee.embedding[middle_word])
end

synthesize_embedding (generic function with 1 method)

In [7]:
word="state"


  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


"state"

In [8]:
function lexical_informed_embeddings(db,ee, word)
    source_wvs = ee.embedding[word]
    target_synsets = Base.flatten((haskey(db.lemmas[pos], word) ? synsets(db, db[pos, word]) : WordNet.Synset[] 
        for pos in keys(WordNet.SYNSET_TYPES))) |> collect
        
    synsets4vectors = Dict{eltype(source_wvs), Synset}()
    for synset in target_synsets
            gloss = filter(s->!ispunct(s), lowercase(synset.gloss))
            context::AbstractArray{SubString{String}} = filter!(w->w!=word, split(gloss))
            wv = synthesize_embedding(ee,context, word)
            synsets4vectors[wv] = synset
    end
    synsets4vectors
end

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


lexical_informed_embeddings (generic function with 1 method)

In [9]:
@time lexical_informed_embeddings(db,ee, "state")

LoadError: LoadError: MethodError: no method matching (::Query.#kw##logprob_of_context)(::Array{Any,1}, ::Query.#logprob_of_context, ::WordEmbeddings.FixedWordSenseEmbedding, ::Array{SubString{String},1}, ::Array{Float32,1})
Closest candidates are:
  #logprob_of_context{S<:String}(::Array{Any,1}, ::Query.#logprob_of_context, ::WordEmbeddings.GenWordEmbedding, !Matched::AbstractArray{S<:String,1}, ::Array{Float32,1})
  #logprob_of_context{S<:String}(::Array{Any,1}, ::Query.#logprob_of_context, !Matched::WordEmbeddings.WordEmbedding, !Matched::AbstractArray{S<:String,1}, !Matched::S<:String)
while loading In[9], in expression starting on line 156

33-element Array{SubString{String},1}:
 "a"             
 "politically"   
 "organized"     
 "body"          
 "of"            
 "people"        
 "under"         
 "a"             
 "single"        
 "government"    
 "the"           
 "has"           
 "elected"       
 "a"             
 "new"           
 "president"     
 "african"       
 "nations"       
 "students"      
 "who"           
 "had"           
 "come"          
 "to"            
 "the"           
 "nations"       
 "capitol"       
 "the"           
 "countrys"      
 "largest"       
 "manufacturer"  
 "an"            
 "industrialized"
 "land"          

In [57]:
?filter!

search: filter! filter Filter PollingFileWatcher fieldtype



```
filter!(function, collection)
```

Update `collection`, removing elements for which `function` is `false`. For associative collections, the function is passed two arguments (key and value).
